In [12]:
import os
from dotenv import load_dotenv, find_dotenv

# from langchain.embeddings import QianfanEmbeddingsEndpoint  # it is deprecated.
# from langchain.chat_models import QianfanChatEndpoint       # it is deprecated.
from langchain_community.embeddings import QianfanEmbeddingsEndpoint
from langchain_community.chat_models import QianfanChatEndpoint
from langchain_openai import ChatOpenAI


def get_model(temperature=0.7):
    if current_llm == 'Qianfan':
        if temperature == 0:
            temperature = 0.01
        model = QianfanChatEndpoint(model="ERNIE-Bot-4", temperature = temperature)
    else:
        model = ChatOpenAI(model="gpt-4", temperature = temperature)
    return model
    
_ = load_dotenv(find_dotenv()) # read local .env file
current_llm = 'Qianfan'

server_name='15.15.174.4'
server_port = 8989  

## Skills

### use getpass 

getpass 是 Python 标准库中的一个模块，用于在命令行接口中安全地输入密码和其他敏感信息。它的主要功能是隐藏用户输入的内容，以保护密码等敏感信息不被显示在屏幕上。

In [1]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

 ········


In [3]:
# print(os.environ["LANGCHAIN_API_KEY"] )

### First Sample



In [9]:
from langchain.chat_models import QianfanChatEndpoint
from langchain_openai import ChatOpenAI  # pip install -qU langchain-openai
from langchain_core.messages import HumanMessage, SystemMessage

model = get_model()


messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="hi!"),
]

print(model.invoke(messages))

messages = [
    SystemMessage(content="Translate the following from English into Chinese"),
    HumanMessage(content="Let's first use the model directly!"),
]

model.invoke(messages)

content='Ciao!' additional_kwargs={'finish_reason': 'normal', 'request_id': 'as-wpakgtxggu', 'object': 'chat.completion', 'search_info': [], 'function_call': {}, 'tool_calls': [{'type': 'function', 'function': {}}]} response_metadata={'token_usage': {'prompt_tokens': 10, 'completion_tokens': 3, 'total_tokens': 13}, 'model_name': 'ERNIE-Bot-4', 'finish_reason': 'normal', 'id': 'as-wpakgtxggu', 'object': 'chat.completion', 'created': 1716278687, 'result': 'Ciao!', 'is_truncated': False, 'need_clear_history': False, 'usage': {'prompt_tokens': 10, 'completion_tokens': 3, 'total_tokens': 13}} id='run-817c5a3c-5004-4908-86ff-d2d4bab7794a-0'


AIMessage(content='让我们先直接使用模型吧！', additional_kwargs={'finish_reason': 'normal', 'request_id': 'as-q8igd9eqsc', 'object': 'chat.completion', 'search_info': [], 'function_call': {}, 'tool_calls': [{'type': 'function', 'function': {}}]}, response_metadata={'token_usage': {'prompt_tokens': 17, 'completion_tokens': 6, 'total_tokens': 23}, 'model_name': 'ERNIE-Bot-4', 'finish_reason': 'normal', 'id': 'as-q8igd9eqsc', 'object': 'chat.completion', 'created': 1716278689, 'result': '让我们先直接使用模型吧！', 'is_truncated': False, 'need_clear_history': False, 'usage': {'prompt_tokens': 17, 'completion_tokens': 6, 'total_tokens': 23}}, id='run-fe84bced-23c1-4143-9666-ec48b2ca8123-0')

#### OutputParsers
想输出为文本

In [10]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

result = model.invoke(messages)
parser.invoke(result)

'让我们先直接使用模型吧！'

In [12]:
chain = model | parser
chain.invoke(messages)

'让我们先直接使用模型吧！'

#### Prompt Templates

使用提示词模版

In [13]:
from langchain.chat_models import QianfanChatEndpoint
from langchain_openai import ChatOpenAI  # pip install -qU langchain-openai
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following into {language}:"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

result = prompt_template.invoke({"language": "italian", "text": "hi"})
result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into italian:'), HumanMessage(content='hi')])

In [14]:
result.to_messages()

[SystemMessage(content='Translate the following into italian:'),
 HumanMessage(content='hi')]

In [15]:
chain = prompt_template | model | parser
chain.invoke({"language": "italian", "text": "hi"})

'ciao'

### Serving with LangServe

LangServe帮助开发人员将LangChain链部署为REST API。

~~~
pip install "langserve[all]"
~~~

参见代码 langserve_sampel.py
~~~
python langserve_sampel.py
~~~

Uvicorn running on http://15.15.174.4:8989。

每个LangServe服务都附带一个简单的内置用户界面，用于配置和调用应用程序，并提供流式输出和中间步骤的可见性。前往 http://localhost:8000/chain/playground/

In [15]:
from langserve import RemoteRunnable

playground_url = f"http://{server_name}:{server_port}/chain/playground/"
print(f'playground_url={playground_url}')

server_url = f"http://{server_name}:{server_port}/chain/"
remote_chain = RemoteRunnable(f"http://{server_name}:{server_port}/chain/")
remote_chain.invoke({"language": "chinese", 
                     "text": "We've learned how to work with language models"})

playground_url=http://15.15.174.4:8989/chain/playground/


'我们已经学会了如何与语言模型合作'

### Build a Chatbot

There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

In [18]:
#!pip install -qU langchain-openai
#!pip install langchain_community

In [53]:
from langchain_core.messages import AIMessage
from langchain_core.messages import HumanMessage

model = get_model()

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

model.invoke([HumanMessage(content="What's my name?")])
# 示例中，是可以识别到我的名字的，这里不知道为啥不行。

AIMessage(content="I'm sorry, as a language model, I don't have access to your personal information, including your name. Your name is a personal privacy matter that you can choose to share with others or not. If you have any other questions to ask, I'll do my best to help.", additional_kwargs={'finish_reason': 'normal', 'request_id': 'as-4f51xccsk0', 'object': 'chat.completion', 'search_info': [], 'function_call': {}, 'tool_calls': [{'type': 'function', 'function': {}}]}, response_metadata={'token_usage': {'prompt_tokens': 6, 'completion_tokens': 61, 'total_tokens': 67}, 'model_name': 'ERNIE-Bot-4', 'finish_reason': 'normal', 'id': 'as-4f51xccsk0', 'object': 'chat.completion', 'created': 1716297856, 'result': "I'm sorry, as a language model, I don't have access to your personal information, including your name. Your name is a personal privacy matter that you can choose to share with others or not. If you have any other questions to ask, I'll do my best to help.", 'is_truncated': False

#### Message History 

In [67]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

model = get_model()
with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [68]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Bob")],
    config=config,
)

response.content

Parent run aecd2d53-f6c2-43a1-b725-0c680e1c90b0 not found for run 2a3290c5-12ea-46b8-994c-998bf8d6faca. Treating as a root run.


"Hello Bob! Nice to meet you. I'm an artificial intelligence language model, and I can answer your questions or chat with you. What can I do for you?"

In [69]:
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

Parent run 240084d2-43f9-4dec-a2f6-ef843f3c5df8 not found for run e5c90d20-61ed-4f5f-a626-583928e53486. Treating as a root run.


"I'm sorry, as a language model, I don't have access to your personal information, including your name. Your name is your personal privacy, and I don't have the ability to obtain it. If you have any other questions to answer or discuss, I'm willing to help."

#### Prompt templates

In [97]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

model = get_model()
chain = prompt | model

response = chain.invoke(
    {"messages": [HumanMessage(content="hi! I'm bob")], "language": "English"}
)

response.content

'Hi Bob! Nice to meet you. How can I assist you today?'

In [96]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc11"}}

response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm todd")], "language": "English"},
    config=config,
)

response.content

Parent run 08fb84d1-c07e-4dea-a187-cb756359490f not found for run 828337db-7b0f-4303-ae5f-0ebc3d2fb663. Treating as a root run.
[ERROR] [05-21 21:46:46] openapi_requestor.py:256 [t:140455267235648]: api request req_id: as-j314gy91ba failed with error code: 336003, err msg: function_call's name can't be blank, please check https://cloud.baidu.com/doc/WENXINWORKSHOP/s/tlmyncueh


APIError: api return error, req_id: as-j314gy91ba code: 336003, msg: function_call's name can't be blank

In [73]:
# 报错 APIError: api return error, req_id: as-ihj57dhpqk code: 336003, msg: function_call's name can't be blank
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "English"},
    config=config,
)

response.content

Parent run 31ebee8d-297e-435d-90d1-ac9eed09f3a1 not found for run bcfe3f6d-d71c-4151-838b-8b709b08b695. Treating as a root run.
[ERROR] [05-21 21:32:28] openapi_requestor.py:256 [t:140455267235648]: api request req_id: as-qth1t0fzph failed with error code: 336003, err msg: function_call's name can't be blank, please check https://cloud.baidu.com/doc/WENXINWORKSHOP/s/tlmyncueh


APIError: api return error, req_id: as-qth1t0fzph code: 336003, msg: function_call's name can't be blank

#### Managing Conversation History

In [98]:
from langchain_core.runnables import RunnablePassthrough


def filter_messages(messages, k=9):
    return messages[-k:]

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)


model = get_model()
chain = (
    RunnablePassthrough.assign(messages=lambda x: filter_messages(x["messages"]))
    | prompt
    | model
)

messages = [ 
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

In [99]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)
response.content

"I'm sorry, as a helpful assistant, I cannot know your name unless you have told me before. If you want to tell me your name, I will do my best to remember it."

In [100]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my fav ice cream")],
        "language": "English",
    }
)
response.content

'You mentioned that you like vanilla ice cream, so I think your favorite ice cream is vanilla flavor.'

In [102]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc20"}}

response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

Parent run 53e0457e-3f79-42b4-b009-75d0f90fa2a1 not found for run dcfd2067-00d0-4a9e-b4bd-fd84c6bd152b. Treating as a root run.


"I'm sorry, as a helpful assistant I don't have access to your personal information. I don't know your name. You can tell me your name, and I'll try to remember it for future interactions."

In [103]:
# 报错
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="whats my favorite ice cream?")],
        "language": "English",
    },
    config=config,
)

response.content

Parent run 746ca598-1a22-4d01-b363-9406093c751d not found for run 49ce2a4f-0703-466a-83b8-f7d127df6858. Treating as a root run.
[ERROR] [05-21 21:50:22] openapi_requestor.py:256 [t:140455267235648]: api request req_id: as-zwfv6g9ijw failed with error code: 336003, err msg: function_call's name can't be blank, please check https://cloud.baidu.com/doc/WENXINWORKSHOP/s/tlmyncueh


APIError: api return error, req_id: as-zwfv6g9ijw code: 336003, msg: function_call's name can't be blank

#### Streaming

In [104]:
config = {"configurable": {"session_id": "abc15"}}
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="hi! I'm todd. tell me a joke")],
        "language": "English",
    },
    config=config,
):
    print(r.content, end="|")

Parent run dbb5353b-a9a1-4b36-837b-7cc9ad8bc5bd not found for run bbfd0062-38c6-4656-a2ac-4646107559e4. Treating as a root run.


Hi Todd|!| Here's a joke for you:

Why did the scarecrow win an award?| Because he was outstanding in his field!|

Hope you enjoyed it!||

# Quickstart

## LLM Chain

In [ ]:
LLM Chain

In [ ]:
#!pip install langchain-openai

In [1]:
import os
import datetime
import qianfan, openai


# from langchain.embeddings.openai import OpenAIEmbeddings
# from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

from langchain.chat_models import QianfanChatEndpoint
from langchain.embeddings import QianfanEmbeddingsEndpoint

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

print(f"QIANFAN_AK={os.environ['QIANFAN_AK']}")
print(f"QIANFAN_SK={os.environ['QIANFAN_SK']}")

openai.api_key = os.environ['OPENAI_API_KEY']
print(f"OPENAI_API_KEY={os.environ['OPENAI_API_KEY']}")

QIANFAN_AK=HCCPsQy5p0Ex1rSEL6oorGQb
QIANFAN_SK=TCVRfPCfbtLr0eDPZRXcywMxcgaNuLDE
OPENAI_API_KEY=ABCDEFGHJLKLMN


In [2]:
# llm = ChatOpenAI()
llm = QianfanChatEndpoint(model="ERNIE-Bot-4")
llm.invoke("how can langsmith help with testing?")

[INFO] [05-08 21:24:22] oauth.py:222 [t:43400]: trying to refresh access_token for ak `HCCPsQ***`
[INFO] [05-08 21:24:26] oauth.py:237 [t:43400]: sucessfully refresh access_token


AIMessage(content='LangSmith是一个强大的工具，可以在多个方面帮助进行测试。以下是LangSmith在测试方面的一些主要功能和应用：\n\n1. 调试和测试：LangSmith可以查看事件链中的每个步骤的模型输入输出，这可以方便地试验新链和新提示，并找到问题的根源，如意外结果、错误或延迟。同时，它可以跟踪数据样本或上传自定义数据集，然后针对数据集运行链和提示，进行手动检查输入输出或者自动化测试。\n2. 评估：LangSmith支持对模型的效果进行量化评估。它可以无缝集成开源评估模块，支持规则评估和LLM自评估，这有助于对模型性能进行全面、客观的衡量。通过评估，开发团队可以了解模型在特定任务上的表现，从而进行针对性的优化。\n3. 监控：LangSmith可以主动跟踪性能指标、模型链性能、调试问题、用户交互体验等，从而持续优化产品。这种监控功能有助于及时发现和解决潜在问题，确保模型的稳定性和可靠性。\n\n总的来说，LangSmith通过提供调试、测试、评估和监控等功能，为开发团队提供了一个统一、高效的测试平台。使用LangSmith，开发团队可以更加便捷地进行模型开发和优化，提高工作效率和产品质量。', additional_kwargs={'finish_reason': 'normal', 'request_id': 'as-1v72w3si6a', 'object': 'chat.completion', 'search_info': [], 'function_call': {}}, response_metadata={'token_usage': {'prompt_tokens': 8, 'completion_tokens': 239, 'total_tokens': 247}, 'model_name': 'ERNIE-Bot-4', 'finish_reason': 'normal', 'id': 'as-1v72w3si6a', 'object': 'chat.completion', 'created': 1715174690, 'result': 'LangSmith是一个强大的工具，可以在多个方面帮助进行测试。以下是LangSmith在测试方面的一些主要功能和应用：\n\n1. 调试和测试：LangSmith可以

In [3]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
])

In [4]:
chain = prompt | llm 
chain.invoke({"input": "how can langsmith help with testing?"})

AIMessage(content='LangSmith是一个功能强大的工具，可以在多个方面帮助进行测试。以下是LangSmith在测试方面的一些主要应用：\n\n1. 调试和测试：LangSmith可以查看事件链中的每个步骤的模型输入输出，这可以方便地试验新链和新提示，找到问题根源，如意外结果、错误或延迟。同时，它还可以跟踪数据样本或上传自定义数据集，然后针对数据集运行链和提示，手动检查输入输出或者进行自动化测试。\n2. 评估：LangSmith支持对模型的效果进行量化评估。它可以无缝集成开源评估模块，支持规则评估和LLM自评估。这有助于对模型的性能进行全面、客观的评估，从而发现模型存在的问题和不足。\n3. 监控和优化：LangSmith可以主动跟踪性能指标、模型链性能、调试问题、用户交互体验等，从而及时发现和解决问题，持续优化产品。此外，LangSmith还可以记录和分析模型应用的中间过程，如提示词等，以便进行更好的优化。\n\n总的来说，LangSmith通过提供调试、测试、评估、监控和优化等功能，可以帮助开发团队更有效地进行测试，提高模型的质量和性能。同时，它还可以降低测试成本，提高测试效率，使团队能够更专注于核心应用的开发和创新。', additional_kwargs={'finish_reason': 'normal', 'request_id': 'as-9aw282m96a', 'object': 'chat.completion', 'search_info': [], 'function_call': {}}, response_metadata={'token_usage': {'prompt_tokens': 18, 'completion_tokens': 247, 'total_tokens': 265}, 'model_name': 'ERNIE-Bot-4', 'finish_reason': 'normal', 'id': 'as-9aw282m96a', 'object': 'chat.completion', 'created': 1715174715, 'result': 'LangSmith是一个功能强大的工具，可以在多个方面帮助进行测试。以下是LangSmith在测试方面的一些主要应用：\n\n1. 调试和测试：La

In [5]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
chain = prompt | llm | output_parser
chain.invoke({"input": "how can langsmith help with testing?"})

'LangSmith是一个功能强大的工具，可以在多个方面帮助进行测试工作。以下是LangSmith在测试方面的一些主要应用：\n\n1. 调试和测试：LangSmith可以查看事件链中的每个步骤的模型输入输出，这可以方便地试验新链和新提示，找到问题根源，如意外结果、错误或延迟。同时，它还可以跟踪延迟和Token使用情况来定位调用性能问题。这些功能使得LangSmith成为一个有效的调试和测试工具。\n2. 数据样本跟踪和自定义数据集上传：LangSmith可以跟踪数据样本或上传自定义数据集，然后针对数据集运行链和提示。这使得测试人员能够使用实际的数据进行测试，并确保系统的性能符合预期。\n3. 自动化测试：除了手动检查输入输出外，LangSmith还支持自动化测试。这可以大大提高测试效率，并减少人为错误的可能性。\n4. 评估模块集成：LangSmith无缝集成开源评估模块，支持规则评估和LLM自评估。这可以帮助测试人员更准确地评估系统的性能，并提供有关如何改进系统的有用信息。\n5. 监控和持续优化：LangSmith可以主动跟踪性能指标、模型链性能、调试问题、用户交互体验等，从而持续优化产品。这意味着测试人员可以使用LangSmith来监控系统的性能，并在必要时进行调整和优化。\n\n总的来说，LangSmith通过提供一系列强大的功能，可以帮助测试人员更有效地进行测试工作，提高系统的质量和性能。'

## Retrieval Chain

In [6]:
#!pip install beautifulsoup4     # beautifulSoup4是一个用于解析HTML和XML文档的Python库。 它使提取数据变得简单，快速且可靠。 它允许您使用简单的Python语法来搜索，修改和导航网页结构
#!pip install faiss-cpu     # Faiss的全称是Facebook AI Similarity Search，是FaceBook针对大规模相似度检索问题开发的一个工具，底层是使用C++代码实现的，提供了python的接口，号称对10亿量级的索引可以做到毫秒级检索。

In [9]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

In [10]:
docs

[Document(page_content="\n\n\n\n\nLangSmith User Guide | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith\n\n\n\n\n\n\n\nSkip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookThis is outdated documentation for \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith, which is no longer actively maintained.For up-to-date documentation, see the latest version.User GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we‚Äôll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they‚Äôre just starting their journey.Prototyping‚ÄãPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and oth

In [7]:
# from langchain_openai import OpenAIEmbeddings
# embeddings = OpenAIEmbeddings()

from langchain.embeddings import QianfanEmbeddingsEndpoint
embeddings =  QianfanEmbeddingsEndpoint()

In [12]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(chunk_size = 384)
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [14]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

llm = QianfanChatEndpoint(model="ERNIE-Bot-4")
document_chain = create_stuff_documents_chain(llm, prompt)

In [15]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'Langsmith can help with testing by allowing users to visualize test results. This capability can provide a more intuitive and understandable way to analyze and interpret test data, which can ultimately lead to improved testing efficiency and effectiveness.'

In [16]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

LangSmith can help with testing by allowing developers to create datasets, which are collections of inputs and reference outputs, and use these to run tests on their LLM applications. This enables developers to quickly test out different prompts and models, and gain an understanding of how their LLM applications are performing in real-world scenarios. By developing an understanding of the types of inputs the app is performing well or poorly on, and how exactly it's breaking down in those cases, LangSmith can assist in the curation of test cases that can help track and improve the performance of the LLM applications.


## Conversation Retrieval Chainm

In [23]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}. Given the above conversation, generate a search query to look up to get information relevant to the conversation"),
    # ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])

print(prompt)

llm = QianfanChatEndpoint(model="ERNIE-Bot-4")
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

input_variables=['chat_history', 'input'] input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]} messages=[MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}. Given the above conversation, generate a search query to look up to get information relevant to the conversation'))]


In [24]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content='a series of interactions between the user and the application. LangSmith provides a threads view that groups traces from a single conversation together, making it easier to track the performance of and annotate your application across multiple turns.Was this page helpful?PreviousQuick StartNextOverviewPrototypingBeta TestingProductionCommunityDiscordTwitterGitHubDocs CodeLangSmith', metadata={'source': 'https://docs.smith.langchain.com/user_guide', 'title': 'LangSmith User Guide | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'LangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we‚Äôll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they‚Äôre just starting their journey.', 'language': 'en'}),
 Document(page_content='These 

In [27]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content='These test cases can be uploaded in bulk, created on the fly, or exported from application traces. LangSmith also makes it easy to run custom evaluations (both LLM and heuristic based) to score test results.Comparison View‚ÄãWhen prototyping different versions of your applications and making changes, it‚Äôs important to see whether or not you‚Äôve regressed with respect to your', metadata={'source': 'https://docs.smith.langchain.com/user_guide', 'title': 'LangSmith User Guide | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'LangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we‚Äôll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize 

## Agent

In [28]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

IndentationError: unexpected indent (521391027.py, line 4)

# Extracting structured outputm

## Quickstart

In [ ]:
#!pip install langchain

# Install a model capable of tool calling
# pip install langchain-openai
# pip install langchain-mistralai
# pip install langchain-fireworks

# Set env vars for the relevant model or load from a .env file:
# import dotenv
# dotenv.load_dotenv()

In [31]:
from typing import Optional

from langchain_core.pydantic_v1 import BaseModel, Field


class Person(BaseModel):
    """Information about a person."""

    # ^ Doc-string for the entity Person.
    # This doc-string is sent to the LLM as the description of the schema Person,
    # and it can help to improve extraction results.

    # Note that:
    # 1. Each field is an `optional` -- this allows the model to decline to extract it!
    # 2. Each field has a `description` -- this description is used by the LLM.
    # Having a good description can help improve extraction results.
    name: Optional[str] = Field(default=None, description="The name of the person")
    hair_color: Optional[str] = Field(
        default=None, description="The color of the peron's hair if known"
    )
    height_in_meters: Optional[str] = Field(
        default=None, description="Height measured in meters"
    )

In [32]:
from typing import Optional

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

# Define a custom prompt to provide instructions and any additional context.
# 1) You can add examples into the prompt template to improve extraction quality
# 2) Introduce additional parameters to take context into account (e.g., include metadata
#    about the document from which the text was extracted.)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm. "
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked to extract, "
            "return null for the attribute's value.",
        ),
        # Please see the how-to about improving performance with
        # reference examples.
        # MessagesPlaceholder('examples'),
        ("human", "{text}"),
    ]
)

In [33]:
from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(model="mistral-large-latest", temperature=0)

runnable = prompt | llm.with_structured_output(schema=Person)

In [34]:
text = "Alan Smith is 6 feet tall and has blond hair."
runnable.invoke({"text": text})

LocalProtocolError: Illegal header value b'Bearer '

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Define a custom prompt to provide instructions and any additional context.
# 1) You can add examples into the prompt template to improve extraction quality
# 2) Introduce additional parameters to take context into account (e.g., include metadata
#    about the document from which the text was extracted.)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm. "
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked "
            "to extract, return null for the attribute's value.",
        ),
        # ↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓
        MessagesPlaceholder("examples"),  # <-- EXAMPLES!
        # ↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑
        ("human", "{text}"),
    ]
)

In [ ]:
from langchain_core.messages import (
    HumanMessage,
)

prompt.invoke(
    {"text": "this is some text", "examples": [HumanMessage(content="testing 1 2 3")]}
)

In [ ]:
from typing import List, Optional

from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI


class Person(BaseModel):
    """Information about a person."""

    # ^ Doc-string for the entity Person.
    # This doc-string is sent to the LLM as the description of the schema Person,
    # and it can help to improve extraction results.

    # Note that:
    # 1. Each field is an `optional` -- this allows the model to decline to extract it!
    # 2. Each field has a `description` -- this description is used by the LLM.
    # Having a good description can help improve extraction results.
    name: Optional[str] = Field(..., description="The name of the person")
    hair_color: Optional[str] = Field(
        ..., description="The color of the peron's eyes if known"
    )
    height_in_meters: Optional[str] = Field(..., description="Height in METERs")


class Data(BaseModel):
    """Extracted data about people."""

    # Creates a model so that we can extract multiple entities.
    people: List[Person]